In [56]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import codecs
from collections import Counter

In [57]:
def load_journos_info():
    journos_file = codecs.open(dirname+'irish_journos_info.txt')
    for line in journos_file:
        parts = line.split('\t')
        journos_info['@'+parts[3]] = str(parts[0])+'\t'+parts[1]+'\t'+parts[2]+'\t'+parts[4].strip()
    journos_file.close()
    return journos_info

In [58]:
#GET RETWEETS PER JOURNALIST
dirname = '/home/clau/Documents/journalists2017_personas/'
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'timeout': 50}])

journos_info = dict()
journos_info = load_journos_info()

for k, v in journos_info.items():
    retweets_file = codecs.open(dirname+'audience_retweets_ireland/'+k.replace('@','')+'.txt','w')
    retweets = set()
    number_retweets = Counter()
    info = v.split('\t')
    journo_id = info[0]
    print('text:'+'\"RT '+k+'\"')
    response = scan(es, index='ireland2017', q='text:'+'\"RT '+k+'\"', scroll='3m')
    for hit in response:
            number_retweets[k] += 1
            if 'retweeted_status' in hit['_source'].keys():
                if 'retweeted_status.quote_count' in hit['_source'].keys(): 
                    if 'retweeted_status.reply_count' in hit['_source'].keys(): 
                        retweets.add(k+'\t'+str(journo_id)+'\t'+hit['_source']['text']+'\t'+str(hit['_source']['retweeted_status']['favorite_count'])+'\t'+str(hit['_source']['retweeted_status']['quote_count'])+'\t'+str(hit['_source']['retweeted_status']['reply_count'])+'\t'+str(hit['_source']['retweeted_status']['retweet_count'])+'\t'+hit['_source']['user']['name']+'\t'+str(hit['_source']['user']['id'])+'\t'+hit['_source']['user']['screen_name']+'\t'+hit['_source']['created_at'])
                    else:
                        retweets.add(k+'\t'+str(journo_id)+'\t'+hit['_source']['text']+'\t'+str(hit['_source']['retweeted_status']['favorite_count'])+'\t'+str(hit['_source']['retweeted_status']['quote_count'])+'\t\t'+str(hit['_source']['retweeted_status']['retweet_count'])+'\t'+hit['_source']['user']['name']+'\t'+str(hit['_source']['user']['id'])+'\t'+hit['_source']['user']['screen_name']+'\t'+hit['_source']['created_at'])
                else:
                    if 'retweeted_status.reply_count' in hit['_source'].keys(): 
                        retweets.add(k+'\t'+str(journo_id)+'\t'+hit['_source']['text']+'\t'+str(hit['_source']['retweeted_status']['favorite_count'])+'\t\t'+str(hit['_source']['retweeted_status']['reply_count'])+'\t'+str(hit['_source']['retweeted_status']['retweet_count'])+'\t'+hit['_source']['user']['name']+'\t'+str(hit['_source']['user']['id'])+'\t'+hit['_source']['user']['screen_name']+'\t'+hit['_source']['created_at'])
                    else:
                        retweets.add(k+'\t'+str(journo_id)+'\t'+hit['_source']['text']+'\t'+str(hit['_source']['retweeted_status']['favorite_count'])+'\t\t\t'+str(hit['_source']['retweeted_status']['retweet_count'])+'\t'+hit['_source']['user']['name']+'\t'+str(hit['_source']['user']['id'])+'\t'+hit['_source']['user']['screen_name']+'\t'+hit['_source']['created_at'])        
            else:
                print('!')
    for r in retweets:
        retweets_file.write(r.replace('&amp;','&')+'\n')
    retweets_file.close()
    print(number_retweets[k])

text:"RT @noriordan"
714
text:"RT @EamonDelaney10"
144
text:"RT @MiguelDelaney"
10544
text:"RT @CaitrionaPerry"
2420
text:"RT @mktighe"
0
text:"RT @shanephelanindo"
30
text:"RT @RMcGreevy1301"
476
text:"RT @fionnansheahan"
30
text:"RT @DrGavinJennings"
52
text:"RT @viviennetraynor"
12
text:"RT @UnaMcCaffreyIT"
4
text:"RT @AlisonHealyIT"
16
text:"RT @FergalBowers"
804
text:"RT @harrymcgee"
186
text:"RT @JoeHumphreys42"
64
text:"RT @SSbike"
!
!
384
text:"RT @LouiseByrneNews"
84
text:"RT @conoreligeorge"
0
text:"RT @judecrosbie"
2
text:"RT @McDonnellDan"
504
text:"RT @kiliandoyle"
24
text:"RT @muirishouston"
48
text:"RT @anthonymurnane"
68
text:"RT @tconnellyRTE"
2682
text:"RT @siobhanbtv3"
44
text:"RT @Colmkelpie"
20
text:"RT @TPeavoy"
6
text:"RT @conorgoodman"
4
text:"RT @TheIrishTim"
6
text:"RT @TWelchTV3"
66
text:"RT @MaryERegan"
672
text:"RT @nathanmurf"
444
text:"RT @AislingRiordan"
0
text:"RT @carrollste"
0
text:"RT @hlinehan"
316
text:"RT @emmetmalone"
120
text:"RT @damien_omeara"

In [59]:
#GET MENTIONS/REPLIES PER JOURNALIST
dirname = '/home/clau/Documents/journalists2017_personas/'
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'timeout': 50}])

journos_info = dict()
journos_info = load_journos_info()

for k, v in journos_info.items():
    retweets_file = codecs.open(dirname+'audience_interactions_ireland/'+k.replace('@','')+'.txt','w')
    retweets = set()
    number_retweets = Counter()
    info = v.split('\t')
    journo_id = info[0]
    print('text:'+'\"RT '+k+'\"')
    response = scan(es, index='ireland2017', q='text:'+'\"'+k+'\"', scroll='3m')
    for hit in response:
            number_retweets[k] += 1
            if 'retweeted_status' in hit['_source'].keys():
                print('Es retweet')
            else:
                retweets.add(k+'\t'+str(journo_id)+'\t'+hit['_source']['text']+'\t'+hit['_source']['user']['name']+'\t'+hit['_source']['user']['screen_name']+'\t'+str(hit['_source']['user']['id'])+'\t'+hit['_source']['created_at']) 
    for r in retweets:
        retweets_file.write(r.replace('&amp;','&')+'\n')
    retweets_file.close()
    print(number_retweets[k])

text:"RT @noriordan"
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
Es retweet
